In [1]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from numba import jit, float64, int64

In [2]:
def make_train_test(data, test_size=0.3):
    sbd = data[data['sbj'] == 0]
    X_train, X_test, y1, y2 = train_test_split(sbd, sbd, test_size=test_size, random_state=1111)
    for i in range(1, data.sbj.max()+1):
        sbd = data[data['sbj'] == i]
        tr, ts, y1, y2 = train_test_split(sbd, sbd, test_size=test_size, random_state=1111)
        X_train = pd.concat([X_train, tr])
        X_test = pd.concat([X_test, ts])
        
    return X_train.reset_index(drop=True), X_test.reset_index(drop=True)

In [3]:
@jit(nopython=True)
def Race(v, b, ndt=0, dt=0.001):
    x = np.array([0.0, 0.0, 0.0])
    rt = 0
    ch = -1
    stop = False
    while not stop:
        x[0] += v[0]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[1] += v[1]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[2] += v[2]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        rt += dt
        if x[0]>= b and x[1]<b and x[2]<b:
            ch = 0.0
            stop = True
        elif x[1]>= b and x[0]<b and x[2]<b:
            ch = 1.0
            stop = True
        elif x[2]>= b and x[0]<b and x[1]<b:
            ch = 2.0
            stop = True
        elif (x[0]>= b and x[1]>=b) or (x[1]>= b and x[2]>=b) or (x[0]>= b and x[2]>=b):
            rt = 0
            x = np.array([0.0, 0.0, 0.0])
    return rt+ndt, ch

In [4]:
data_df = pd.read_csv('../../_00_data/Kr2011_prep.csv', index_col=0)

min_rt = data_df.groupby(by='sbj')['rt'].min()

In [5]:
tr_df, ts_df = make_train_test(data_df, test_size=0.4)

obs_train = np.asarray(tr_df[['rt', 'choice_0', 'choice_1', 'choice_2', 'sbj']])
X_train = np.asarray(tr_df[['item_value_0', 'item_value_1', 'item_value_2', 
                            'gaze_0', 'gaze_1', 'gaze_2']])

obs_test = np.asarray(ts_df[['rt', 'choice_0', 'choice_1', 'choice_2', 'sbj']])
X_test = np.asarray(ts_df[['item_value_0', 'item_value_1', 'item_value_2', 
                           'gaze_0', 'gaze_1', 'gaze_2']])

print(X_train.shape, X_test.shape)

(1741, 6) (1181, 6)


In [6]:
def fpt(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(b/(2*np.pi*t**3)**(1/2) * keras.backend.exp(-1/(2*t) * (v*t-b)**2)) + tf.cast(tf.math.less_equal(t, 0), tf.float32)*0.1**10

def phi(x):
    return 0.5*tf.math.erf(x/tf.sqrt(2.0)) + 0.5

def cdf(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(phi((v*t-b)/tf.sqrt(t)) + tf.exp(2*v*b)*phi((-v*t-b)/tf.sqrt(t))) + (tf.cast(tf.math.less_equal(t, 0), tf.float32)*0)

def G2(data, pred):
    
    sbj_indices = tf.cast(data[:, 4], tf.int32)
    
    # Gather values directly using TensorFlow operations
    gathered_min_rt = tf.gather(min_rt, sbj_indices)
    sigmoid_weights = tf.gather(model.layers[-1].weights[-1], sbj_indices)
    ndt = gathered_min_rt * tf.math.sigmoid(sigmoid_weights)

    thr_weights = tf.gather(model.layers[-1].weights[-2], sbj_indices)
    thr = tf.math.log(1 + tf.exp(thr_weights))

    scl_weights = tf.gather(model.layers[-1].weights[-3], sbj_indices)
    scl = tf.math.log(1 + tf.exp(scl_weights))
    
    ll1 = (obs_train[:, 1]) * fpt(obs_train[:, 0]-ndt, thr, scl*pred[:, 0]) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 1])) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 2]))
    ll2 = (obs_train[:, 2]) * fpt(obs_train[:, 0]-ndt, thr, scl*pred[:, 1]) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 0])) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 2]))
    ll3 = (obs_train[:, 3]) * fpt(obs_train[:, 0]-ndt, thr, scl*pred[:, 2]) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 0])) * (1.0 - cdf(obs_train[:, 0]-ndt, thr, scl*pred[:, 1]))
    
    return -2*tf.reduce_sum(tf.math.log(tf.maximum(ll1 + ll2 + ll3, 1e-14)), axis=-1)

In [7]:
class Out_layer(keras.layers.Layer):
    def __init__(self, units=3, input_dim=32, n_sbj_prms=10):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="uniform", trainable=True)
        
        self.scaling = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.threshold = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.non_decision_time = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)

    def call(self, inputs):
        return tf.math.log(1 + tf.exp(tf.matmul(inputs, self.w) + self.b))
    
def make_model(layers_neuron, input_shape, output_shape, activation, n_sbj_prms):
    keras.backend.clear_session()

    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    for l in layers_neuron:
        model.add(layers.Dense(l, activation=activation))
    
    model.add(Out_layer(units=output_shape, input_dim=layers_neuron[-1], n_sbj_prms=n_sbj_prms))

    print(model.summary())
    
    return model

In [8]:
layers_neuron = [10, 10, 10, 10]
input_shape = 6
output_shape = 3
activation = 'sigmoid'
n_sbj = tr_df.sbj.nunique()

model = make_model(layers_neuron, input_shape, output_shape, activation, n_sbj)

model.build()
model.compile(optimizer="RMSprop", loss=G2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                70        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 10)                110       
                                                                 
 out_layer (Out_layer)       (None, 3)                 123       
                                                                 
Total params: 523 (2.04 KB)
Trainable params: 523 (2.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [9]:
sbj_indices = tf.cast(obs_train[:, 4], tf.int32)
obs_train = tf.cast(obs_train, tf.float32)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.cast(X_train, tf.float32)

min_rt = tf.convert_to_tensor(min_rt)
min_rt = tf.cast(min_rt, tf.float32)

In [10]:
history = model.fit(X_train, 
                    obs_train, 
                    shuffle=False, 
                    batch_size=X_train.shape[0], 
                    epochs=5000)

Epoch 1/5000
1/1 [==============================] - 1s 915ms/step - loss: 17004.9375
Epoch 2/5000
1/1 [==============================] - 0s 3ms/step - loss: 16879.9102
Epoch 3/5000
1/1 [==============================] - 0s 4ms/step - loss: 16790.9551
Epoch 4/5000
1/1 [==============================] - 0s 4ms/step - loss: 16717.4688
Epoch 5/5000
1/1 [==============================] - 0s 4ms/step - loss: 16652.9473
Epoch 6/5000
1/1 [==============================] - 0s 3ms/step - loss: 16594.3887
Epoch 7/5000
1/1 [==============================] - 0s 3ms/step - loss: 16540.1172
Epoch 8/5000
1/1 [==============================] - 0s 3ms/step - loss: 16489.1055
Epoch 9/5000
1/1 [==============================] - 0s 4ms/step - loss: 16440.6562
Epoch 10/5000
1/1 [==============================] - 0s 3ms/step - loss: 16394.2773
Epoch 11/5000
1/1 [==============================] - 0s 3ms/step - loss: 16349.6162
Epoch 12/5000
1/1 [==============================] - 0s 3ms/step - loss: 16306.3955

1/1 [==============================] - 0s 4ms/step - loss: 13804.0762
Epoch 99/5000
1/1 [==============================] - 0s 4ms/step - loss: 13782.3037
Epoch 100/5000
1/1 [==============================] - 0s 4ms/step - loss: 13760.6729
Epoch 101/5000
1/1 [==============================] - 0s 4ms/step - loss: 13739.1904
Epoch 102/5000
1/1 [==============================] - 0s 3ms/step - loss: 13717.8506
Epoch 103/5000
1/1 [==============================] - 0s 4ms/step - loss: 13696.6553
Epoch 104/5000
1/1 [==============================] - 0s 3ms/step - loss: 13675.6035
Epoch 105/5000
1/1 [==============================] - 0s 4ms/step - loss: 13654.6924
Epoch 106/5000
1/1 [==============================] - 0s 3ms/step - loss: 13633.9258
Epoch 107/5000
1/1 [==============================] - 0s 3ms/step - loss: 13613.2979
Epoch 108/5000
1/1 [==============================] - 0s 3ms/step - loss: 13592.8105
Epoch 109/5000
1/1 [==============================] - 0s 3ms/step - loss: 13572.4

1/1 [==============================] - 0s 2ms/step - loss: 12265.9736
Epoch 195/5000
1/1 [==============================] - 0s 3ms/step - loss: 12254.5381
Epoch 196/5000
1/1 [==============================] - 0s 2ms/step - loss: 12243.1699
Epoch 197/5000
1/1 [==============================] - 0s 2ms/step - loss: 12231.8711
Epoch 198/5000
1/1 [==============================] - 0s 2ms/step - loss: 12220.6396
Epoch 199/5000
1/1 [==============================] - 0s 2ms/step - loss: 12209.4746
Epoch 200/5000
1/1 [==============================] - 0s 3ms/step - loss: 12198.3760
Epoch 201/5000
1/1 [==============================] - 0s 3ms/step - loss: 12187.3438
Epoch 202/5000
1/1 [==============================] - 0s 2ms/step - loss: 12176.3740
Epoch 203/5000
1/1 [==============================] - 0s 3ms/step - loss: 12165.4707
Epoch 204/5000
1/1 [==============================] - 0s 2ms/step - loss: 12154.6309
Epoch 205/5000
1/1 [==============================] - 0s 2ms/step - loss: 12143.

1/1 [==============================] - 0s 3ms/step - loss: 11401.8740
Epoch 291/5000
1/1 [==============================] - 0s 3ms/step - loss: 11394.6533
Epoch 292/5000
1/1 [==============================] - 0s 3ms/step - loss: 11387.4600
Epoch 293/5000
1/1 [==============================] - 0s 5ms/step - loss: 11380.2910
Epoch 294/5000
1/1 [==============================] - 0s 4ms/step - loss: 11373.1455
Epoch 295/5000
1/1 [==============================] - 0s 3ms/step - loss: 11366.0283
Epoch 296/5000
1/1 [==============================] - 0s 2ms/step - loss: 11358.9336
Epoch 297/5000
1/1 [==============================] - 0s 2ms/step - loss: 11351.8623
Epoch 298/5000
1/1 [==============================] - 0s 2ms/step - loss: 11344.8135
Epoch 299/5000
1/1 [==============================] - 0s 3ms/step - loss: 11337.7891
Epoch 300/5000
1/1 [==============================] - 0s 3ms/step - loss: 11330.7891
Epoch 301/5000
1/1 [==============================] - 0s 3ms/step - loss: 11323.

1/1 [==============================] - 0s 2ms/step - loss: 10796.3271
Epoch 387/5000
1/1 [==============================] - 0s 2ms/step - loss: 10790.7275
Epoch 388/5000
1/1 [==============================] - 0s 2ms/step - loss: 10785.1396
Epoch 389/5000
1/1 [==============================] - 0s 2ms/step - loss: 10779.5596
Epoch 390/5000
1/1 [==============================] - 0s 2ms/step - loss: 10773.9941
Epoch 391/5000
1/1 [==============================] - 0s 4ms/step - loss: 10768.4385
Epoch 392/5000
1/1 [==============================] - 0s 3ms/step - loss: 10762.8936
Epoch 393/5000
1/1 [==============================] - 0s 3ms/step - loss: 10757.3604
Epoch 394/5000
1/1 [==============================] - 0s 3ms/step - loss: 10751.8389
Epoch 395/5000
1/1 [==============================] - 0s 2ms/step - loss: 10746.3262
Epoch 396/5000
1/1 [==============================] - 0s 2ms/step - loss: 10740.8252
Epoch 397/5000
1/1 [==============================] - 0s 2ms/step - loss: 10735.

...

1/1 [==============================] - 0s 4ms/step - loss: 5379.8926
Epoch 4616/5000
1/1 [==============================] - 0s 3ms/step - loss: 5378.7529
Epoch 4617/5000
1/1 [==============================] - 0s 5ms/step - loss: 5378.2866
Epoch 4618/5000
1/1 [==============================] - 0s 4ms/step - loss: 5378.1030
Epoch 4619/5000
1/1 [==============================] - 0s 4ms/step - loss: 5375.8369
Epoch 4620/5000
1/1 [==============================] - 0s 4ms/step - loss: 5380.4194
Epoch 4621/5000
1/1 [==============================] - 0s 4ms/step - loss: 5378.7520
Epoch 4622/5000
1/1 [==============================] - 0s 4ms/step - loss: 5436.5723
Epoch 4623/5000
1/1 [==============================] - 0s 4ms/step - loss: 5382.6880
Epoch 4624/5000
1/1 [==============================] - 0s 4ms/step - loss: 5387.1763
Epoch 4625/5000
1/1 [==============================] - 0s 5ms/step - loss: 5377.5527
Epoch 4626/5000
1/1 [==============================] - 0s 4ms/step - loss: 5381.4

1/1 [==============================] - 0s 4ms/step - loss: 5372.7485
Epoch 4712/5000
1/1 [==============================] - 0s 3ms/step - loss: 5375.9604
Epoch 4713/5000
1/1 [==============================] - 0s 3ms/step - loss: 5371.9937
Epoch 4714/5000
1/1 [==============================] - 0s 5ms/step - loss: 5429.1577
Epoch 4715/5000
1/1 [==============================] - 0s 3ms/step - loss: 5431.7065
Epoch 4716/5000
1/1 [==============================] - 0s 4ms/step - loss: 5377.6221
Epoch 4717/5000
1/1 [==============================] - 0s 5ms/step - loss: 5432.2651
Epoch 4718/5000
1/1 [==============================] - 0s 4ms/step - loss: 5380.1919
Epoch 4719/5000
1/1 [==============================] - 0s 3ms/step - loss: 5379.0508
Epoch 4720/5000
1/1 [==============================] - 0s 3ms/step - loss: 5375.0747
Epoch 4721/5000
1/1 [==============================] - 0s 5ms/step - loss: 5374.9458
Epoch 4722/5000
1/1 [==============================] - 0s 5ms/step - loss: 5373.6

1/1 [==============================] - 0s 4ms/step - loss: 5428.1426
Epoch 4808/5000
1/1 [==============================] - 0s 4ms/step - loss: 5372.2690
Epoch 4809/5000
1/1 [==============================] - 0s 4ms/step - loss: 5370.8203
Epoch 4810/5000
1/1 [==============================] - 0s 4ms/step - loss: 5372.2485
Epoch 4811/5000
1/1 [==============================] - 0s 5ms/step - loss: 5371.6309
Epoch 4812/5000
1/1 [==============================] - 0s 4ms/step - loss: 5375.0449
Epoch 4813/5000
1/1 [==============================] - 0s 4ms/step - loss: 5374.0664
Epoch 4814/5000
1/1 [==============================] - 0s 5ms/step - loss: 5375.5200
Epoch 4815/5000
1/1 [==============================] - 0s 4ms/step - loss: 5378.7100
Epoch 4816/5000
1/1 [==============================] - 0s 3ms/step - loss: 5378.7749
Epoch 4817/5000
1/1 [==============================] - 0s 4ms/step - loss: 5369.4277
Epoch 4818/5000
1/1 [==============================] - 0s 4ms/step - loss: 5370.5

1/1 [==============================] - 0s 4ms/step - loss: 5421.4814
Epoch 4904/5000
1/1 [==============================] - 0s 5ms/step - loss: 5428.5532
Epoch 4905/5000
1/1 [==============================] - 0s 4ms/step - loss: 5414.8101
Epoch 4906/5000
1/1 [==============================] - 0s 4ms/step - loss: 5368.1729
Epoch 4907/5000
1/1 [==============================] - 0s 4ms/step - loss: 5365.4238
Epoch 4908/5000
1/1 [==============================] - 0s 4ms/step - loss: 5365.9810
Epoch 4909/5000
1/1 [==============================] - 0s 4ms/step - loss: 5369.5137
Epoch 4910/5000
1/1 [==============================] - 0s 4ms/step - loss: 5370.1284
Epoch 4911/5000
1/1 [==============================] - 0s 3ms/step - loss: 5369.1562
Epoch 4912/5000
1/1 [==============================] - 0s 4ms/step - loss: 5367.6123
Epoch 4913/5000
1/1 [==============================] - 0s 4ms/step - loss: 5377.6724
Epoch 4914/5000
1/1 [==============================] - 0s 4ms/step - loss: 5431.2

1/1 [==============================] - 0s 4ms/step - loss: 5368.2261
Epoch 5000/5000
1/1 [==============================] - 0s 4ms/step - loss: 5368.6025


In [11]:
pred_v = model.predict(X_test)
threshold = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-2])))
ndt = np.asanyarray(np.asanyarray(min_rt)*tf.math.sigmoid(model.layers[-1].weights[-1]))

37/37 [==============================] - 0s 596us/step


In [12]:
pred_bhv = {'sbj': [],
            'rt': [],
            'ch': [],
            'value_diff': [],
            'gaze_diff': []}

scaling = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-3])))
scaling = np.asanyarray([scaling[i] for i in ts_df['sbj']])

for n in range(20):
    for s in range(ts_df.shape[0]):
        rt, ch = Race(scaling[s]*pred_v[s,:], threshold[ts_df.sbj[s]], ndt[ts_df.sbj[s]])
        pred_bhv['sbj'].append(ts_df.sbj[s])
        pred_bhv['rt'].append(rt)
        pred_bhv['ch'].append(ch)
        pred_bhv['value_diff'].append(ts_df['Right Value - Mean Other Value'][s])
        pred_bhv['gaze_diff'].append(ts_df['Dwell Time Advantage (R - O)'][s])
        
pred_bhv = pd.DataFrame(pred_bhv)
pred_bhv['right'] = pd.get_dummies(pred_bhv['ch'])[2]

In [13]:
pred_bhv['right'] = pd.get_dummies(pred_bhv['ch'])[2]
pred_bhv.to_csv('Kr2011_out_of_sample.csv')
ts_df.to_csv('Kr2011_test.csv')